In [1]:
import copy
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pprint
from scipy.optimize import linprog
from pytest import approx

from mex.simplex.simplex_networks import create_matrix, pivots_col, pivots_row, find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot
from mex.simplex.problem_definition import add_cons, constrain, add_obj, obj, maxz, minz

## Problema prototipo:

$$\min_{x \in \mathbb{R}^2} 100x_1 + 50x_2 + 100x_3 $$

$$\text{sujeto a: }$$
$$ x_1 + x_2 + x_3 = 3000 $$
$$ 28x_1 + 14x_2 + 10x_3 \leq 42000 $$
$$ 10x_1 + 12x_2 + 6x_3 \leq 24000 $$
$$ 30x_1 + 20x_2 + 30x_3 \geq 75000 $$
$$ 10x_1 + 10x_2 + 15x_3 \geq 36000 $$

## Resolvemos con scipy

In [44]:
A_ub = np.array([[28, 14, 10], [10, 12, 6], [-30, -20, -30], [-10, -10, -15]])
b_ub = np.array([42000, 24000, -75000, -36000])
A_eq = np.array([[1, 1, 1]])
b_eq = np.array([3000])
c = np.array([100, 50, 100])

In [45]:
net_obj = linprog(c, A_ub, b_ub, A_eq, b_eq, bounds=(0, None)).fun

In [46]:
net_coeff_obj = linprog(c, A_ub, b_ub, A_eq, b_eq, bounds=(0, None)).x

In [47]:
net_obj

249999.98785527673

In [48]:
net_coeff_obj

array([1.21456692e-06, 9.99999949e+02, 1.99999990e+03])

## Resolvemos con nuestro paquete

In [49]:
n_var_approx = 5
n_cons_approx = 3

In [50]:
matrix_net_approx = create_matrix(n_var_approx, n_cons_approx)

In [51]:
constrain(matrix_net_approx,'1,1,1,E,3000')
constrain(matrix_net_approx,'28,14,10,L,42000')
constrain(matrix_net_approx,'10,12,6,L,24000')
constrain(matrix_net_approx,'-30,-20,-30,L,-75000')
constrain(matrix_net_approx,'-10,-10,-15,L,-36000')
obj(matrix_net_approx,'100,50,100,0')

In [53]:
problem_approx = minz(matrix_net_approx)

In [54]:
net_approx = problem_approx['min']
problem_approx.pop('min')
net_coeff_approx = np.array(list(problem_approx.values()))

In [58]:
net_approx

0.0

In [ ]:
net_coeff_appro

In [55]:
assert net_obj == approx(net_approx)
assert np.round(net_coeff_obj,0) == approx(net_coeff_approx)

AssertionError: 

In [56]:
net_approx

0.0

In [57]:
net_coeff_approx

array([   0.,    0.,    0., 3000., 3000.])